Модуль wave

1. Чтение

In [ ]:
import wave
import struct

f = wave.open("cta0001.wav")
# f = wave.open("cta0001_stereo.wav")

num_samples = f.getnframes()
print(num_samples)
samplerate = f.getframerate()
sampwidth = f.getsampwidth()
num_channels = f.getnchannels()

fmt = str(num_samples * num_channels) + ("h" if sampwidth == 2 else "c")  # на практике обычно 2 байта на отсчёт
# полную таблицу соответствий см. здесь: https://docs.python.org/3/library/struct.html

signal = struct.unpack(fmt, f.readframes(num_samples * num_channels))

In [ ]:
# stereo file
left = signal[::2]
right = signal[1::2]
left == right

In [ ]:
import matplotlib.pyplot as plt
plt.plot(signal)
plt.show()

2. Запись

In [4]:
from math import sin

samplerate = 22050
sampwidth = 2
num_channels = 1
num_samples = samplerate * 2
fmt = str(num_samples) + ("h" if sampwidth == 2 else "c")

ampl = 2 ** 14
sine = [int(ampl * sin(x / 20)) for x in range(num_samples)]
signal = struct.pack(fmt, *sine)

f = wave.open("output.wav", "wb")
f.setnchannels(num_channels)
f.setsampwidth(sampwidth)
f.setframerate(samplerate)
f.writeframes(signal)
f.close()

3. Файлы без заголовка (.sbl)

In [ ]:
samplerate = 22050
sampwidth = 2
num_channels = 1

with open("cta0001.sbl", "rb") as f:
    raw_signal = f.read()
    num_samples = len(raw_signal) // sampwidth
    fmt = str(num_samples) + ("h" if sampwidth == 2 else "c")
    signal = struct.unpack(fmt, raw_signal)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(signal)
plt.show()

Модуль scipy.io.wavfile
1. Чтение

In [ ]:
import scipy.io.wavfile as wav
samplerate, signal = wav.read("cta0001.wav")
# samplerate, signal = wav.read("cta0001_stereo.wav")
print(signal.shape)

2. Запись

In [7]:
import numpy as np

samplerate = 22050
num_samples = samplerate * 2

ampl = 2 ** 14
sine = np.array([int(ampl * sin(x / 20)) for x in range(num_samples)], dtype=np.int16)

wav.write("output2.wav", samplerate, sine)

Модуль wavio

In [ ]:
!pip install wavio

1. Чтение

In [ ]:
import wavio
data = wavio.read("n1fpt1.wav")

signal, samplerate, sampwidth = data.data, data.rate, data.sampwidth

2. Запись

In [ ]:
samplerate = 22050
num_samples = samplerate * 2

ampl = 2 ** 14
sine = np.array([int(ampl * sin(x / 20)) for x in range(num_samples)], dtype=np.int16)

wavio.write("output3.wav", sine, samplerate)
# wavio.write("output3.wav", sine, samplerate, sampwidth=2)

Работа с массивами numpy (которые возвращают scipy и wavio)

In [ ]:
new_array = np.asarray([1, 2, 3, 4, 5], dtype=np.int16) # используем int16 для работы с 16-битными wav-файлами
print(new_array)
print(new_array.shape)

In [ ]:
new_2d_array = np.asarray([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10]], dtype=np.int16) # так должны выглядеть массивы, содержащие данные из стереофайлов
print(new_2d_array)
print(new_2d_array.shape)

Сделаем из двух одномерных массивов двумерный:

In [ ]:
array_2d = np.asarray((new_array, new_array), dtype=np.int16).T # T - это транспонирование
print(array_2d)
print(array_2d.shape)

In [ ]:
zeros_array = np.zeros(5, dtype=np.int16) # массив, заполненный нулями
print(zeros_array.shape)

In [ ]:
zeros_array2d = np.zeros((5, 2), dtype=np.int16)
print(zeros_array2d.shape)

In [ ]:
print(new_array)
new_array = np.flip(new_array)
print(new_array)

In [ ]:
big_array = np.concatenate((new_2d_array, zeros_array, new_2d_array))
print(big_array)

Будьте внимательны! При чтении моно-файлов scipy возвращает одномерный массив, wavio - двумерный, у которого второе измерение равно 1.  
Конвертация из одной формы в другую:

In [ ]:
array2d = np.asarray([[1], [2], [3], [4], [5]])
print(array2d.shape)
array1d = np.squeeze(array2d)
print(array1d.shape)

In [ ]:
array1d = np.asarray([1, 2, 3, 4, 5])
print(array1d.shape)
array2d = np.reshape(array1d, (-1, 1))
print(array2d.shape)

Задание:

1. Считать файл .wav
2. В правый канал положить отсчёты левого в обратном порядке.
3. Вставить паузы 200 мс (или любое другое число, но явно прописанное в коде) на 1/4, 1/2 и 3/4 длительности.
4. Записать в новый файл.